In [1]:
import numpy as np
import random

# Toy Problem
t = np.array([[1,1,0,0,1,0], [1,0,0,1,1,1], [0,0,1,1,1,1], [1,1,1,1,1,1], [0,0,0,1,1,1], [1,1,1,1,0,0]])
sens = [3,2,2,1]
n = len(sens)

#sol = [(0,0), (0,4), (3,4), (4,2)]

# Predefined variables
max_gens = 100 # maximun number of generations
pop_size = 50 # number of particles
max_vel = 2 # maximum velocity
c1, c2 = 2.0, 2.0
neighborhood = 2

In [2]:
def disp_symbol(t,sol=False):
    if sol:
        if(t == 1 and sol != 0):
            return sol
        elif(t == 0 and sol != 0):
            return "#"
        else:
            return "-"
    else:
        if(t == 0):
            return "X"
        else:
            return "-"

def display_solution(t,sol,sens):
    n = len(t)
    print("  ",end='')
    for i in range(1,n +1):
        print(i,end=" ")
    print("               ",end='')
    for i in range(1,n +1):
        print(i,end=" ")
    print()
    for y in range(0,n):
        print(y+1,end=' ')
        for x in range(0,n):
            print(disp_symbol(t[y][x]),end=' ')
        if((y+1) == int(n/2)):
            print("     ==>     ",end='')
        else:
            print("             ",end='')
        print(y+1,end=' ')
        for x in range(0,n):
            print(disp_symbol(t[y][x],gen_sol_t(sol,sens,t,True)[y][x]),end=' ')
        print()
#sol = [(0,0), (0,4), (3,4), (4,3)]
#display_solution(t,sol,sens)

In [75]:
class Particle:
    def __init__(self,t,sens,sol):
        self.t = t
        self.sens = sens
        self.sol = sol # [(),(),(),etc...]
        self.fit = fev(t,sens,sol)
        self.vel =[]
        for i,sens in enumerate(self.sens):
            self.vel.append(random.randint(-max_vel,max_vel))
            #self.vel.append(8)
        self.best = self
    def __repr__(self):
        return '{}: {} FEV: {} V: {}'.format(self.__class__.__name__,
                                  self.sol,
                                  self.fit,
                                  self.vel)

    # Move The Particle to the best neighbour every sensor will follow the best neighbour direction as its destiny,
    # moving according to each sensor velocity (each sensor inside the particl may have a different velocity and a different direction).
    def move(self):
        sol = self.sol
        velocity = self.vel
        best = self.best

        if best.sol == sol:
            self.sol = sol
        else:
            actual = np.array(sol)
            obj = np.array(best.sol)
            for i in range(len(sol)):
                dx = obj[i][0] - actual[i][0]
                dy = obj[i][1] - actual[i][1]
                sx = dx/abs(dx)
                sy = dy/abs(dy)

                vel = abs(velocity[i])

                if vel == 0:
                    svel = 0
                else:
                    svel = vel/velocity[i]
                newx = actual[i][0]
                newy = actual[i][1]

                if dx==0 and dy==0:
                    pass
                else:
                    while vel > 0:
                        if dx == 0:
                            pass
                        elif newx == 0 and svel*sx == -1:
                            vel-=1
                        else:
                            newx += svel*sx
                            vel -=1
                            if vel == 0 or (self.sens[i]+newx) == n or (newx == 0 and newy==0):
                                break

                        if dy == 0:
                            pass
                        elif newy == 0 and svel*sy == -1:
                            vel-=1
                        else:
                            newy += svel*sy
                            vel -=1
                            if vel == 0 or (self.sens[i]+newy) == n or (newy == 0 and newx==0):
                                break

                        if dx==0 and dy==0:
                            break

                new = [int(newx), int(newy)]
                if check_position(sens[i],  tuple(new), n):
                    self.sol[i] = tuple(new)


        self.update_fit()

    def update_fit(self):
        self.fit = fev(self.t,self.sens,self.sol)

    def update_best(self,part):
        if (self.fit >= part.fit):
            self.best = part

    def upadte_Neighbor(self,rankedParticles,index):
        limit = pop_size
        neighbs = self.fit
        
        for i in range(neighborhood):
            nextN = index + i
            if(nextN >= limit):
                nextN = nextN - limit
            if(rankedParticles[nextN].fit > neighbs):
                self.best = rankedParticles[nextN]
                neighbs = rankedParticles[nextN].fit
            prevoiusN = index - i
            
            if(prevoiusN < 0):
                prevoiusN = prevoiusN + limit
                
            if(rankedParticles[prevoiusN].fit > neighbs):
                self.best = rankedParticles[prevoiusN]
                neighbs = rankedParticles[prevoiusN].fit

    # Calculate the velocity of each sensor internally 
    def update_velocity(self,c1,c2,gbest):
        for i,sen in enumerate(self.sens):
            for j in range(0,1): # In the tuple 0 and 1 (0,0) of each sensor
                v1 = c1 * random.random() * (self.best.sol[i][j] - self.sol[i][j])
                v2 = c2 * random.random() * (gbest.sol[i][j] - self.sol[i][j])
                new_vel = self.vel[i] + int(v1) + int(v2)
                if new_vel >  max_vel : self.vel[i] =  max_vel
                if new_vel < -max_vel : self.vel[i] = -max_vel


In [30]:
#Checks if its valid the position
def check_position(sensor,pos,size):
    x = pos[0]+sensor-1
    y = pos[1]+sensor-1
    cont = True
    if(x > (size - 1) or y > (size - 1)):
        cont = False
    return cont

# gen_population(sensors, number of the population, board size)
def gen_population(sens,n,size):
    population = []
    for i in range(n):
        localSol = []
        for se in sens:
            cont = True
            pos = (0,0)
            while cont:
                x = random.randint(0,size - 1)
                y = random.randint(0,size - 1)
                pos = (x,y)
                cont = not check_position(se,pos,size)
            localSol.append(pos)
        population.append(Particle(t,sens,localSol))
    return population


In [2]:
# Calculate the fit for each particle
def getKey(part):
    part.update_fit()
    return part.fit

# Ranks each particle from best to worst
def rank(pop):
    sortedPopulation = sorted(pop, key=getKey, reverse=True)
    return sortedPopulation
#rank(gen_population(sens,10,6))

In [29]:
# Generates the solution terrain gen_sol_t(solution, sensors)
def gen_sol_t(sol,sens,t,numbers=False):
    gen_sol = np.zeros_like(t)
    for i, sen in enumerate(sens):
        pos = sol[i]
        for x in range(pos[0],pos[0] + sen):
            for y in range(pos[1],pos[1] + sen):
                if numbers:
                    gen_sol[x][y] = i + 1
                else:
                    gen_sol[x][y] = 1
    return np.array(gen_sol)

# fev(terrain, sensors, solution)
def fev(t,sens,sol):
    return np.sum(np.multiply(t,gen_sol_t(sol,sens,t)))

#sol = [(0,0), (0,4), (3,4), (4,2)]
#fev(t,sens,sol)

In [73]:
def run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood):
    pop = gen_population(sens,pop_size,n)
    all_bests = []
    for i in range(max_gens):
        rankedParticles = rank(pop)
        gbest = rankedParticles[0]
        all_bests.append(gbest)
        index = 0
        for part in rankedParticles:
            part.update_best(gbest)
            part.upadte_Neighbor(rankedParticles,index)
            index = index +1
        newPop = []
        for i,part in enumerate(rankedParticles):
            part.move()
            part.update_velocity(c1,c2,gbest)
            newPop.append(part)
        pop = newPop
    bestParticles = rank(all_bests)
    print(bestParticles[0])
    display_solution(t,rankedParticles[0].sol,sens)
    print()

In [8]:
# Toy Problem
t = np.array([[1,1,0,0,1,0], [1,0,0,1,1,1], [0,0,1,1,1,1], [1,1,1,1,1,1], [0,0,0,1,1,1], [1,1,1,1,0,0]])
sens = [3,2,2,1]
n = 6

#sol = [(0,0), (0,4), (3,4), (4,2)]

# Predefined variables
max_gens = 20 # maximun number of generations
pop_size = 1000 # number of particles
max_vel = 2 # maximum velocity
c1, c2 = 2.0, 1.5
neighborhood = 2
run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood)

/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in long_scalars
/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in long_scalars


Particle: [(3, 1), (3, 3), (1, 4), (5, 1)] FEV: 11 V: [-2, 0, 1, 2]
  1 2 3 4 5 6                1 2 3 4 5 6 
1 - - X X - X              1 - - X X - X 
2 - X X - - -              2 - X X - 3 3 
3 X X - - - -      ==>     3 X X - - 3 3 
4 - - - - - -              4 - 1 1 2 2 - 
5 X X X - - -              5 X # # 2 2 - 
6 - - - - X X              6 - 4 1 1 X X 



In [17]:
t = np.array([[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,1,1,0,0],
              [0,0,0,0,0,1,1,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0]])
sens = [2]
n = len(t)

# Predefined variables
max_gens = 100 # maximun number of generations
pop_size = 500 # number of particles
max_vel = 1 # maximum velocity
c1, c2 = 1.0, 3.0
neighborhood = 100
run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood)

/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in long_scalars
/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in long_scalars


Particle: [(1, 5)] FEV: 0 V: [1]
  1 2 3 4 5 6 7 8 9                1 2 3 4 5 6 7 8 9 
1 X X X X X X X X X              1 X X X X X X X X X 
2 X X X X X X X X X              2 X X X X X # # X X 
3 X X X X X X X X X              3 X X X X X # # X X 
4 X X X X X X X X X      ==>     4 X X X X X X X X X 
5 X X X X X X X X X              5 X X X X X X X X X 
6 X X X X X - - X X              6 X X X X X - - X X 
7 X X X X X - - X X              7 X X X X X - - X X 
8 X X X X X X X X X              8 X X X X X X X X X 
9 X X X X X X X X X              9 X X X X X X X X X 



# Problems

Pop = 100| Gen = 50 | max_vel = 1 | C1 = 1 | C2 = 1

In [77]:
# Toy Problem
t = np.array([[1,1,0,0,1,0],
              [1,0,0,1,1,1],
              [0,0,1,1,1,1],
              [1,1,1,1,1,1],
              [0,0,0,1,1,1],
              [1,1,1,1,0,0]])
sens = [3,2,2,1]
n = len(t)

# Predefined variables
max_gens = 50 # maximun number of generations
pop_size = 100 # number of particles
max_vel = 1 # maximum velocity
c1, c2 = 1.0, 1.0
neighborhood = 100
run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood)


/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in long_scalars
/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in long_scalars


Particle: [(1, 3), (2, 1), (0, 0), (3, 0)] FEV: 16 V: [0, 1, 0, 1]
  1 2 3 4 5 6                1 2 3 4 5 6 
1 - - X X - X              1 3 3 X X - X 
2 - X X - - -              2 3 # X 1 1 1 
3 X X - - - -      ==>     3 X # 2 1 1 1 
4 - - - - - -              4 4 2 2 1 1 1 
5 X X X - - -              5 X X X - - - 
6 - - - - X X              6 - - - - X X 



In [78]:
# Random Problem
t = np.array([[1,1,0,0,1,0,1,1,1],
              [1,0,0,1,1,1,1,1,1],
              [0,0,1,1,1,1,0,0,1],
              [1,1,1,1,1,1,1,1,1],
              [0,0,0,1,1,1,1,1,1],
              [1,1,1,1,0,0,1,1,1],
              [1,1,0,1,1,0,0,0,1],
              [1,1,0,1,1,0,0,0,1],
              [1,1,1,1,1,0,0,0,1]])

sens = [3,2,2,3,1]
n = len(t)



# Predefined variables
max_gens = 50 # maximun number of generations
pop_size = 100 # number of particles
max_vel = 1 # maximum velocity
c1, c2 = 1.0, 1.0
neighborhood = 100
run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood)


/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in long_scalars
/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in long_scalars


Particle: [(3, 6), (5, 0), (7, 0), (2, 3), (8, 8)] FEV: 27 V: [1, 1, 1, 1, 1]
  1 2 3 4 5 6 7 8 9                1 2 3 4 5 6 7 8 9 
1 - - X X - X - - -              1 - - X X - X - - - 
2 - X X - - - - - -              2 - X X - - - - - - 
3 X X - - - - X X -              3 X X - 4 4 4 X X - 
4 - - - - - - - - -      ==>     4 - - - 4 4 4 1 1 1 
5 X X X - - - - - -              5 X X X 4 4 4 1 1 1 
6 - - - - X X - - -              6 2 2 - - X X 1 1 1 
7 - - X - - X X X -              7 2 2 X - - X X X - 
8 - - X - - X X X -              8 3 3 X - - X X X - 
9 - - - - - X X X -              9 3 3 - - - X X X 5 



In [80]:
# Solitaire Problem
t = np.array([[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,1,1,0,0],
              [0,0,0,0,0,1,1,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0]])
sens = [2]
n = len(t)

# Predefined variables
max_gens = 50 # maximun number of generations
pop_size = 100 # number of particles
max_vel = 1 # maximum velocity
c1, c2 = 1.0, 1.0
neighborhood = 100
run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood)

/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in long_scalars
/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in long_scalars


Particle: [(5, 5)] FEV: 4 V: [0]
  1 2 3 4 5 6 7 8 9                1 2 3 4 5 6 7 8 9 
1 X X X X X X X X X              1 X X X X X X X X X 
2 X X X X X X X X X              2 X X X X X X X X X 
3 X X X X X X X X X              3 X X X X X X X X X 
4 X X X X X X X X X      ==>     4 X X X X X X X X X 
5 X X X X X X X X X              5 X X X X X X X X X 
6 X X X X X - - X X              6 X X X X X 1 1 X X 
7 X X X X X - - X X              7 X X X X X 1 1 X X 
8 X X X X X X X X X              8 X X X X X X X X X 
9 X X X X X X X X X              9 X X X X X X X X X 



In [84]:
# PERFECT MATCH
t = np.array([[1,1,1,0,0,0,0,0],
              [1,1,1,0,1,1,1,0],
              [1,1,1,0,1,1,1,0],
              [0,0,0,0,1,1,1,0],
              [0,0,0,1,0,0,0,0],
              [1,0,0,0,0,1,1,0],
              [0,0,0,0,0,1,1,0],
              [1,0,0,0,0,0,0,0]])
sens = [1,1,1,2,3,3]

n=len(t)

# Predefined variables
max_gens = 50 # maximun number of generations
pop_size = 100 # number of particles
max_vel = 1 # maximum velocity
c1, c2 = 1.0, 1.0
neighborhood = 100
run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood)

/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in long_scalars
/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in long_scalars


Particle: [(7, 0), (4, 3), (7, 0), (5, 5), (0, 0), (1, 4)] FEV: 24 V: [-1, 1, 0, 1, 1, 1]
  1 2 3 4 5 6 7 8                1 2 3 4 5 6 7 8 
1 - - - X X X X X              1 5 5 5 X X X X X 
2 - - - X - - - X              2 5 5 5 X 6 6 6 X 
3 - - - X - - - X              3 5 5 5 X 6 6 6 X 
4 X X X X - - - X      ==>     4 X X X X 6 6 6 X 
5 X X X - X X X X              5 X X X 2 X X X X 
6 - X X X X - - X              6 - X X X X 4 4 X 
7 X X X X X - - X              7 X X X X X 4 4 X 
8 - X X X X X X X              8 3 X X X X X X X 



In [89]:
# Big MATCH
t = np.array([[1,0,1,0,1,0,1,0,1,0],
              [0,1,1,0,0,1,1,0,1,1],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,1,1,1,0,0,0,1,1],
              [1,1,1,1,1,1,1,1,1,1],
              [0,0,1,1,1,0,1,0,0,1],
              [0,0,1,1,1,0,1,0,1,1],
              [0,1,0,1,0,1,1,1,0,1],
              [1,0,0,1,0,0,1,0,0,1],
              [1,0,0,0,0,0,0,0,0,0]])
sens = [5,4,3,2,1,1]
n = len(t)

# Predefined variables
max_gens = 50 # maximun number of generations
pop_size = 100 # number of particles
max_vel = 1 # maximum velocity
c1, c2 = 1.0, 1.0
neighborhood = 100
run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood)


/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in long_scalars
/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in long_scalars


Particle: [(3, 5), (3, 1), (0, 0), (7, 0), (4, 0), (3, 7)] FEV: 36 V: [1, 1, 1, 1, 1, 1]
  1 2 3 4 5 6 7 8 9 10                1 2 3 4 5 6 7 8 9 10 
1 - X - X - X - X - X              1 3 # 3 X - X - X - X 
2 X - - X X - - X - -              2 # 3 3 X X - - X - - 
3 X X X X X X X X X X              3 # # # X X X X X X X 
4 X X - - - X X X - -              4 X # 2 2 2 # # # 1 1 
5 - - - - - - - - - -      ==>     5 5 2 2 2 2 1 1 1 1 1 
6 X X - - - X - X X -              6 X # 2 2 2 # 1 # # 1 
7 X X - - - X - X - -              7 X # 2 2 2 # 1 # 1 1 
8 X - X - X - - - X -              8 # 4 X - X 1 1 1 # 1 
9 - X X - X X - X X -              9 4 # X - X X - X X - 
10 - X X X X X X X X X              10 - X X X X X X X X X 



Pop = 200| Gen = 100 | max_vel = 3 | C1 = 2 | C2 = 3

In [90]:
# Toy Problem
t = np.array([[1,1,0,0,1,0],
              [1,0,0,1,1,1],
              [0,0,1,1,1,1],
              [1,1,1,1,1,1],
              [0,0,0,1,1,1],
              [1,1,1,1,0,0]])
sens = [3,2,2,1]
n = len(t)

# Predefined variables
max_gens = 100 # maximun number of generations
pop_size = 200 # number of particles
max_vel = 3 # maximum velocity
c1, c2 = 2.0, 3.0
neighborhood = 100
run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood)


/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in long_scalars
/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in long_scalars


Particle: [(2, 3), (0, 0), (0, 3), (1, 5)] FEV: 16 V: [3, 0, 2, -3]
  1 2 3 4 5 6                1 2 3 4 5 6 
1 - - X X - X              1 2 2 X # 3 X 
2 - X X - - -              2 2 # X 3 3 4 
3 X X - - - -      ==>     3 X X - 1 1 1 
4 - - - - - -              4 - - - 1 1 1 
5 X X X - - -              5 X X X 1 1 1 
6 - - - - X X              6 - - - - X X 



In [95]:
# Random Problem
t = np.array([[1,1,0,0,1,0,1,1,1],
              [1,0,0,1,1,1,1,1,1],
              [0,0,1,1,1,1,0,0,1],
              [1,1,1,1,1,1,1,1,1],
              [0,0,0,1,1,1,1,1,1],
              [1,1,1,1,0,0,1,1,1],
              [1,1,0,1,1,0,0,0,1],
              [1,1,0,1,1,0,0,0,1],
              [1,1,1,1,1,0,0,0,1]])

sens = [3,2,2,3,1]
n = len(t)

# Predefined variables
max_gens = 100 # maximun number of generations
pop_size = 200 # number of particles
max_vel = 3 # maximum velocity
c1, c2 = 2.0, 3.0
neighborhood = 100
run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood)


/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in long_scalars
/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in long_scalars


Particle: [(2, 3), (7, 0), (7, 3), (3, 6), (8, 8)] FEV: 27 V: [1, 3, 3, 3, -3]
  1 2 3 4 5 6 7 8 9                1 2 3 4 5 6 7 8 9 
1 - - X X - X - - -              1 - - X X - X - - - 
2 - X X - - - - - -              2 - X X - - - - - - 
3 X X - - - - X X -              3 X X - 1 1 1 X X - 
4 - - - - - - - - -      ==>     4 - - - 1 1 1 4 4 4 
5 X X X - - - - - -              5 X X X 1 1 1 4 4 4 
6 - - - - X X - - -              6 - - - - X X 4 4 4 
7 - - X - - X X X -              7 - - X - - X X X - 
8 - - X - - X X X -              8 2 2 X 3 3 X X X - 
9 - - - - - X X X -              9 2 2 - 3 3 X X X 5 



In [96]:
# Solitaire Problem
t = np.array([[0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,1,1,0,0],
              [0,0,0,0,0,1,1,0,0],
              [0,0,0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0,0,0]])
sens = [2]
n = len(t)

# Predefined variables
max_gens = 100 # maximun number of generations
pop_size = 200 # number of particles
max_vel = 3 # maximum velocity
c1, c2 = 2.0, 3.0
neighborhood = 100
run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood)



/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in long_scalars
/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in long_scalars


Particle: [(5, 5)] FEV: 4 V: [3]
  1 2 3 4 5 6 7 8 9                1 2 3 4 5 6 7 8 9 
1 X X X X X X X X X              1 X X X X X X X X X 
2 X X X X X X X X X              2 X X X X X X X X X 
3 X X X X X X X X X              3 X X X X X X X X X 
4 X X X X X X X X X      ==>     4 X X X X X X X X X 
5 X X X X X X X X X              5 X X X X X X X X X 
6 X X X X X - - X X              6 X X X X X 1 1 X X 
7 X X X X X - - X X              7 X X X X X 1 1 X X 
8 X X X X X X X X X              8 X X X X X X X X X 
9 X X X X X X X X X              9 X X X X X X X X X 



In [102]:
# PERFECT MATCH
t = np.array([[1,1,1,0,0,0,0,0],
              [1,1,1,0,1,1,1,0],
              [1,1,1,0,1,1,1,0],
              [0,0,0,0,1,1,1,0],
              [0,0,0,1,0,0,0,0],
              [1,0,0,0,0,1,1,0],
              [0,0,0,0,0,1,1,0],
              [1,0,0,0,0,0,0,0]])
sens = [1,1,1,2,3,3]

n=len(t)

# Predefined variables
max_gens = 100 # maximun number of generations
pop_size = 200 # number of particles
max_vel = 3 # maximum velocity
c1, c2 = 2.0, 3.0
neighborhood = 100
run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood)


/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in long_scalars
/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in long_scalars


Particle: [(0, 4), (7, 0), (4, 3), (6, 5), (1, 4), (0, 0)] FEV: 22 V: [3, 3, 3, 3, 1, -2]
  1 2 3 4 5 6 7 8                1 2 3 4 5 6 7 8 
1 - - - X X X X X              1 6 6 6 X # X X X 
2 - - - X - - - X              2 6 6 6 X 5 5 5 X 
3 - - - X - - - X              3 6 6 6 X 5 5 5 X 
4 X X X X - - - X      ==>     4 X X X X 5 5 5 X 
5 X X X - X X X X              5 X X X 3 X X X X 
6 - X X X X - - X              6 - X X X X - - X 
7 X X X X X - - X              7 X X X X X 4 4 X 
8 - X X X X X X X              8 2 X X X X # # X 



In [103]:
# Big MATCH
t = np.array([[1,0,1,0,1,0,1,0,1,0],
              [0,1,1,0,0,1,1,0,1,1],
              [0,0,0,0,0,0,0,0,0,0],
              [0,0,1,1,1,0,0,0,1,1],
              [1,1,1,1,1,1,1,1,1,1],
              [0,0,1,1,1,0,1,0,0,1],
              [0,0,1,1,1,0,1,0,1,1],
              [0,1,0,1,0,1,1,1,0,1],
              [1,0,0,1,0,0,1,0,0,1],
              [1,0,0,0,0,0,0,0,0,0]])
sens = [5,4,3,2,1,1]
n = len(t)

# Predefined variables
max_gens = 100 # maximun number of generations
pop_size = 200 # number of particles
max_vel = 3 # maximum velocity
c1, c2 = 2.0, 3.0
neighborhood = 100
run(t,sens,n,max_gens,pop_size,c1,c2,neighborhood)


/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:31: RuntimeWarning: invalid value encountered in long_scalars
/ext/anaconda5/lib/python3.6/site-packages/ipykernel/__main__.py:30: RuntimeWarning: invalid value encountered in long_scalars


Particle: [(4, 5), (4, 1), (1, 7), (3, 4), (9, 0), (1, 2)] FEV: 35 V: [2, 1, 3, 3, 3, 1]
  1 2 3 4 5 6 7 8 9 10                1 2 3 4 5 6 7 8 9 10 
1 - X - X - X - X - X              1 - X - X - X - X - X 
2 X - - X X - - X - -              2 X - 6 X X - - # 3 3 
3 X X X X X X X X X X              3 X X X X X X X # # # 
4 X X - - - X X X - -              4 X X - - 4 # X # 3 3 
5 - - - - - - - - - -      ==>     5 - 2 2 2 4 4 1 1 1 1 
6 X X - - - X - X X -              6 X # 2 2 2 # 1 # # 1 
7 X X - - - X - X - -              7 X # 2 2 2 # 1 # 1 1 
8 X - X - X - - - X -              8 X 2 # 2 # 1 1 1 # 1 
9 - X X - X X - X X -              9 - X X - X # 1 # # 1 
10 - X X X X X X X X X              10 5 X X X X X X X X X 

